https://dsmonk.medium.com/training-and-deploying-of-quantized-llms-with-lora-and-gptq-part-2-2-ec7b54659c9e

Потребность памяти 11Gb, 13Gb для датасетя 5000/1024

Обучение на диалогах.

Обратить внимание на ###end

Разбирать ответ по тегам, хорошо работает вопрос + Response. В этом случае ловить надо end of response


watch -d -n 0.5 nvidia-smi

In [ ]:
!pip install -q -U transformers peft accelerate optimum

In [ ]:
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu117/

In [ ]:
!pip install trl

In [1]:
# Empty VRAM cache
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [2]:
#device_string = "cuda:" + str(local_rank)
#kwargs["device_map"] = device_string

In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"]="false"

#from huggingface_hub import login
#login(token=os.getenv("HUGGINGFACE_TOKEN"))

# Чужой датасет

In [4]:
from datasets import Dataset
from datasets import load_dataset
#train_ds, test_ds = load_dataset("knkarthick/dialogsum", split=["train", "test[0:200]"])
#dataset = Dataset.from_pandas(train_df)

In [ ]:
train_ds

In [ ]:
import pandas as pd

# convert to pandas
train_df = pd.DataFrame(train_ds)
test_df = pd.DataFrame(test_ds)
train_df.head(2)

In [ ]:
# %%
# instruction finetuning data preparation function

# тестовый датасет идет без ### End

def prepare_dataset(df, split="train"):
    text_col = []
    instruction = """Write a concise summary of the below input text.Ensure that responses covers the key points of the text.Only provide full sentence responses."""  # change instuction according to the task
    if split == "train":
        for _, row in df.iterrows():
            input_q = row["dialogue"]
            output = row["summary"]
            text = (
                "### Instruction: \n"
                + instruction
                + "\n### Input: \n"
                + input_q
                + "\n### Response :\n"
                + output
                + "\n### End"
            )  # keeping output column in training dataset
            text_col.append(text)
        df.loc[:, "text"] = text_col
    else:
        for _, row in df.iterrows():
            input_q = row["dialogue"]
            text = (
                "### Instruction: \n"
                + instruction
                + "\n### Input: \n"
                + input_q
                + "\n### Response :\n"
            )  # not keeping output column in test dataset
            text_col.append(text)
        df.loc[:, "text"] = text_col
    return df

train_df = prepare_dataset(train_df, "train")
test_df = prepare_dataset(test_df, "test")
print(train_df.iloc[0].text)

# My Dataset

In [ ]:
import json
with open('srv/result_dataset.json', 'r') as f:
    j=json.load(f)
import pandas as pd
df=pd.DataFrame(j)
df.sample(2)

In [ ]:
def prepare_dataset(df):
    text_col = []
    instruction = """Ответь на вопрос. """  # change instuction according to the task
    for _, row in df.iterrows():
        input_q = row["question"]
        output = row["answer"]
        text = (
            "### Instruction: \n"
            + instruction
            + "\n### Input: \n"
            + input_q
            + "\n### Response :\n"
            + output
            + "\n### End"
            )  # keeping output column in training dataset
        text_col.append(text)
    df.loc[:, "text"] = text_col
    return df

train_df = prepare_dataset(df)
print(train_df.iloc[0].text)

# Small INK dataset

In [5]:
import pandas as pd
df=pd.read_csv('10K_ink_dataset.txt', sep='|')
df.sample(3)

,instruction,output,input
2487,NaN,Наименование Групп (Папок) нижнего уровня соот...,NaN
3928,NaN,21. Отчет «Контроль исполнения заявок системой...,NaN
4385,NaN,Рисунок 13. Заявка на оплату – Бивалютный плат...,NaN


In [ ]:
df.loc[464]

In [6]:
df=df[:464]

In [7]:
import re
def prepare(row, instruction='Ответь на вопрос максимально точно и лаконично.'):
    if pd.notna(row['instruction']):
        return f"""### Instruction:
{instruction}

### Input:
{row['instruction']}

### Response:
{row['output']}

### End"""
    else:
        # по поводу End в строке надо проверить конечно
        return re.sub('\t', ' ', row['output'])+"\n### End"

df['text']=df.apply(lambda x: prepare(x), axis=1)
df.sample(3)

,instruction,output,input,text
115,1C: УПП ИНК. PM.CostEngineering,Модуль в составе УПП. Решение PM.cost engineer...,NaN,### Instruction:\nОтветь на вопрос максимально...
444,1. Какие периоды доступны для ручного редактир...,В заявке по бюджету движения денежных средств ...,NaN,### Instruction:\nОтветь на вопрос максимально...
267,2. Какая форма доступна при нажатии на кнопку ...,"Форма ""Акт приема-передачи давальческих матери...","В документе ""Передача материалов между подрядч...",### Instruction:\nОтветь на вопрос максимально...


In [8]:
df.drop(columns=['instruction', 'output', 'input'], inplace=True)

In [9]:
df.drop_duplicates()

,text
0,### Instruction:\nОтветь на вопрос максимально...
1,### Instruction:\nОтветь на вопрос максимально...
2,### Instruction:\nОтветь на вопрос максимально...
3,### Instruction:\nОтветь на вопрос максимально...
4,### Instruction:\nОтветь на вопрос максимально...
...,...
459,### Instruction:\nОтветь на вопрос максимально...
460,### Instruction:\nОтветь на вопрос максимально...
461,### Instruction:\nОтветь на вопрос максимально...
462,### Instruction:\nОтветь на вопрос максимально...


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464 entries, 0 to 463
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    464 non-null    object
dtypes: object(1)
memory usage: 3.8+ KB


In [11]:
train_df=df.copy()

# Подготовка и обучение

In [12]:
from datasets import Dataset
dataset = Dataset.from_pandas(train_df)

In [13]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training
from transformers import GPTQConfig

In [14]:
model_id = "TheBloke/Llama-2-7B-GPTQ"
#model_id='TheBloke/Llama-2-13B-GPTQ'

In [15]:
quantization_config_loading = GPTQConfig(bits=8, use_exllama=False) #disable_exllama=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id, quantization_config=quantization_config_loading, device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model.config.use_cache = False
model.config.pretraining_tp = 1

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. use_exllama, exllama_config, use_cuda_fp16, max_input_length) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


In [16]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

from peft import LoraConfig, get_peft_model
config = LoraConfig(
    r= 16, #8,
    lora_alpha=16,
    target_modules=["k_proj","o_proj","q_proj","v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [17]:
model = get_peft_model(model, config)
model.print_trainable_parameters()
# %%
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

# needed for llama 2 tokenizer
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
args=TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1, #4,# чем больше, тем быстрее учится
        num_train_epochs=7,
        warmup_steps=2,
        max_steps=-1,
        learning_rate=2e-4,
        fp16=True, #use mixed precision training
        logging_steps=10, #увеличить
        output_dir="INK_small",
        overwrite_output_dir=True,
        save_steps=20,
        save_total_limit=2,
        optim="adamw_hf",
        save_strategy="epoch",
        report_to="none")

trainable params: 16,777,216 || all params: 279,187,456 || trainable%: 6.009301506726721


In [18]:
from trl import SFTTrainer
# use_reentrant=True
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset,
    peft_config=config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=1024,
)

/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


Map:   0%|          | 0/464 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [19]:
train_result = trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.818000
20,1.215200
30,1.084800
40,1.023000
50,1.018900
60,1.141700
70,1.011800
80,0.992400
90,0.942600
100,1.083700


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

# Сохранение адаптера

In [20]:
# %%
checkpoint_name ="INK_small"
#to merge and save the model
output_dir = os.path.join(args.output_dir, checkpoint_name)
trainer.model.save_pretrained(output_dir)

# Чтение и запуск адаптера

In [22]:
output_dir="INK_small/INK_small"

In [23]:
from peft import AutoPeftModelForCausalLM
import torch

# To perform inference on the test dataset example load the model from the checkpoint
persisted_model = AutoPeftModelForCausalLM.from_pretrained(
    output_dir,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda",
)

model_id = "TheBloke/Llama-2-7B-GPTQ"
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)
# needed for llama 2 tokenizer
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [49]:
iloc=240
q=df.loc[iloc, 'text']
print(q)
#print(df.loc[iloc, 'answer'])

### Instruction:
Ответь на вопрос максимально точно и лаконично.

### Input:
2. Какие данные можно увидеть в нижней части обработки «Карта номерного фонда»?

### Response:
- В нижней части обработки можно увидеть информацию о проживающем: ФИО, место работы, должность, подразделение, 
номер заявки или брони на указанный период, даты заезда и выезда, статус, а также комментарии из карточки. По 
двойному клику на поле "Проживающий" можно перейти в карточку проживающего.

### End

In [61]:
#Какие кнопки используются для групповой обработки задач на форме списка задач?
q='Какие рекомендации по решению ситуации с выводом вагонов без внешнего суточного задания?'

In [66]:
#inference on test data example
from time import perf_counter
from rich import print
from transformers import GenerationConfig
#text = test_df['text'][4]
text=f'<s>### Instruction: Ответь на вопрос максимально точно и лаконично.\n\n### Input: {q}\n\n### Response:'
inputs = tokenizer(text, return_tensors="pt").to('cuda')
generation_config = GenerationConfig(
    penalty_alpha=1, 
    do_sample = True, 
    top_k=10, 
    temperature=0.3, 
    repetition_penalty=1.2,
    max_new_tokens=200
)
start_time = perf_counter()
outputs = persisted_model.generate(**inputs, generation_config=generation_config, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
end_time = perf_counter()
output_time = end_time - start_time
print(f"Time taken for inference: {round(output_time,2)} seconds")

### Instruction: Ответь на вопрос максимально точно и лаконично.

### Input: Какие рекомендации по решению ситуации с выводом вагонов без внешнего суточного задания?

### Response: Рекомендация 1: Если причиной выхода вагонов без внешнего задания является отсутствие связей в 
таблице «Выгрузка/Связь», то необходимо установить все новые связи, которые были бы установлены после запуска 
заданий для этого дня (временная дата согласуется с текущей датой). Рекомендация 2: После того, как будут 
установлены все новые связи, следует перезаписать целительную группу из командной области и пересоздать задание 
«Дневное распределение».

### End:
Окончание

### EndResponse:
Закрывающее сообщение

### EndInput:
Конечный инPUT

### EndInstruction:
Концовый INSTRУ

Time taken for inference: 9.32 seconds

In [65]:
text = q+"\nОтвет:"
inputs = tokenizer(text, return_tensors="pt").to('cuda')
generation_config = GenerationConfig(
    penalty_alpha=1, 
    do_sample = True, 
    top_k=2, 
    temperature=0.2, 
    repetition_penalty=1.5,
    max_new_tokens=400
)
start_time = perf_counter()
outputs = persisted_model.generate(**inputs, generation_config=generation_config, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
end_time = perf_counter()
output_time = end_time - start_time
print(f"Time taken for inference: {round(output_time,2)} seconds")

Какие рекомендации по решению ситуации с выводом вагонов без внешнего суточного задания?
Ответ: Регистрировать причинение и освобождение путей под управлением, а также запуска/остановку движения в 
регистре «Собственные события» (кнопка «Записать»). После этого необходимо перезагрузиться системой. При этом 
система автоматически удалила все добавленное после последней версии файлы константы 
REGION_VARIABLES\VL-DATA\STATION\_DEPOT and VL-DATA\STAFF\_EMPLOYEE, что может повлияет на работу системы..

Time taken for inference: 6.44 seconds